# Boost Switching regulator LM3478
- Supply: 20 V
- Output voltage: 40 V (theoretically up to 45 V)
	- Make the output voltage variable through some variable resistor (Manually / voltage controlled)

In [ ]:

#############! DESIGN PARAMETERS ##################

### Take from LTSpice simulation + add on top
I_rms_A = 0.5 # Amps (slightly overdimensioned already)
I_peak_A = 1 # Amps, for each bridge spike of 1 amp 180 degrees shifted

### DIODE Choice:
# 40 V max
# 1 A RMS current max
V_diode = 0.55 # SS14
V_max_diode = 20 # 

### MOSFET Choice: AO3400A-mosfet
Vfet_gsth = 1.45 # Volt
Vfet_diode = 1 # Volt
Rfet_dsn = 50e-3 # At 3 amps, 2.5 V
Vfet_dsn = Rfet_dsn * I_rms_A
print(f"Vfet_dsn {Vfet_dsn}")

# Required Output Voltage
Vout = 5.5